# Coordle Idea
1. Get query, e.g. "What is love?"
2. Tokenize query
3. Create a vector using word2vec of the tokens by summing them, or finding the average or whatever.
   lets call it the query vector. 
4. Find sentence vectors that are the closesest to the query vector.
5. Return the papers corresponding to said sentence vectors.

# Coordle Frontend
💩

# Coordle Backend
Essentially need a good way to do kNN search. 


In [1]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
import en_core_sci_lg # Biomedical word embeddings
from utils import clean_text, load_pickle, save_pickle
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque

import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

In [2]:
df = pd.read_csv('data/cord-19-data.csv')

In [3]:
df.columns

Index(['cord_uid', 'paper_id', 'source', 'is_pmc', 'title', 'body_text', 'doi',
       'pubmed_id', 'license', 'abstract', 'publish_time', 'authors',
       'journal', 'url', 'language'],
      dtype='object')

In [4]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, logs_filename: str):
        self.output_dir = output_dir
        self.prefix = prefix
        self.logs_filename = logs_filename
        self.epoch = 0

    def on_epoch_end(self, model):
        cum_loss = model.get_latest_training_loss()
        if self.epoch == 0:
            loss = cum_loss
        else:
            loss = cum_loss - self.loss_previous_step
        self.loss_previous_step = loss
        with open(join_path(self.output_dir, self.logs_filename), 'a+') as file:
            file.write(f'Epoch #{self.epoch}, loss: {loss}\n')
        
        output_path = join_path(self.output_dir, f'{self.prefix}_epoch_{self.epoch}.model')
        model.save(output_path)
        self.epoch += 1    

In [5]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec', 'w2v_model_epoch_29.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [6]:
W = model.trainables.syn1neg

In [22]:
# %%time
import coordle_backend
reload(coordle_backend)
from coordle_backend import SentVectorDoc, Index

docsample = df.iloc[0]
def test_SentVectorDoc():
    doc = SentVectorDoc(W, int_to_word, word_to_int, docsample['cord_uid'], 
                        store_sents=True)
    doc, _ = doc.fit(docsample['body_text'])
    print(doc.tf_idf_score)
    return doc

def test_Index():
    coordle = Index(W, int_to_word, word_to_int)
    for i in tqdm(range(64), position=0):
        sample = df.iloc[i]
        coordle.add(sample['cord_uid'], sample['title'], sample['body_text'])
    results = coordle['retarded white woman']
    for result in results:
        print(result.tf_idf_score)

test_Index()
# fuck = test_SentVectorDoc()

100%|██████████| 64/64 [00:03<00:00, 20.83it/s]

Query tokens:  ['retarded', 'white', 'woman']
0.0020807697679692933
0.00018694061058895325
0.0011374975837968187
0.0007151166249038057
0.00032058572116838686
0.00036932555454032554
0.0006353045015886488
0.0009191689133383365
0.0010971925513000812
0.0007226398505643675
0.000599337424750022
0.0004915835508397247
0.0004532295530124071
0.0005395245456743806
0.0008292929119973831
0.001972422822980034
0.0002603536704058114
0.00015819876990291387
0.00021578544950075153
0.0004344583171630591
0.0009664510826594685
